Chapter 5. Universal Quantification (Forall)
==================

Universal quantification is another core concept in **Prove-It**.  A `Forall` operation, formatted with the $\forall$ symbol, is used to represent universal quantification.  For example, $\forall_x P(x)$ means that $P(x)$ is true for any instance of $x$.  $P(x)$ holds true universally over instances of $x$.  Like `Implies`, `Forall` is a core concept that is aliased at the top proveit level but is defined outside of the core in the `proveit.logic` package. It is known in the core for use in the *specialization* and *generalization* derivation steps discussed below.  First, let us example the `Forall` class

In [1]:
# Let us create a basic Forall expression and examine it
from proveit import Forall
from proveit.common import x, P, Px, Q, Qx, R, Rx, S
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)
basicForallExpr

forall_{x in S | Q(x),R(x)} P(x)

The meaning of this **Expression** is that $P(x)$ is a true statement for all instances of $x$ for which $x \in S$ and $Q(x)$ is true.  Let us examine the internal structure of this expression.

In [2]:
basicForallExpr.exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x00000000045683C8>

`Forall` derives from `OperationOverInstances` (`proveit._generic_.operation_over_instances.OperationOverInstances` aliased as `proveit.OperationOverInstances`) which defines the **NamedExpression** structures with `domain`, `imap` (short for "instance mapping"), `conds` (short for "conditions"), and `iexpr` (short for "instance expression") sub-**Expression** roles.  This is a useful structure for all of the `OperationOverInstances` type of expressions ($\exists$, $\sum$, $\prod$, etc., as well as $\forall$).  In our example above, `imap` defines the universal quantification instances using a **Lambda** map, mapping $x$ (the "instance variable") to the `iexpr`, $P(x)$, as well as $x$-dependent `conditions`, $(Q(x), R(x))$.  The  `domain` over which $x$ is being quantified is $S$.  Each of these `Forall` parts may be accessed as follows:

In [3]:
basicForallExpr.instanceVars # list of Variables

[x]

In [4]:
basicForallExpr.instanceExpr

P(x)

In [5]:
basicForallExpr.conditions

(Q(x),R(x))

In [6]:
basicForallExpr.domain

S

Specialization
======

The *specialization* derivation step uses *expression substitution* internally.  The difference is that *specialization* has proof implications and enforces the extra restrictions to justify these proof implications.  It also eliminates one or more of the outer $\forall$ operations.


### Basic Specialization

Let us take our basic/generic example of the `Forall` expression above and specialize it with a particular "instance" expression.  To do so, we will make assumptions to trivially allow this derivation step to be taken (just to show how this works).

In [7]:
from proveit import Operation, InSet, ExpressionList
from proveit.common import fy
assumptions = ExpressionList(basicForallExpr, InSet(fy, S), Operation(Q, fy), Operation(R, fy))
assumptions

([forall_{x in S | Q(x),R(x)} P(x)],(f(y) in S),Q(f(y)),R(f(y)))

`InSet` is another core concept that is defined outside of the core in `proveit.logic`.  It represents the set membership operation using the $\in$ symbol.  It is needed as a core concept specifically for the purpose of ensuring that universal quantification requirements are met (the "instance" expression must be "in" the domain set).

In [8]:
basicForallSpec = basicForallExpr.specialize({x:fy}, assumptions=assumptions)
basicForallSpec

{[forall_{x in S | Q(x),R(x)} P(x)],(f(y) in S),Q(f(y)),R(f(y))} |= P(f(y))

We have proven, somewhat trivially, that $P(f(y))$ is true assuming that $\forall_{x \in S~|~Q(x)} P(x)$, $f(y) \in S$, $Q(f(y))$ are all true statements.  Let is take a look at the proof for this statement.

In [9]:
basicForallSpec.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004568E08>

This indicates that the proof requires a *specialization* step and explicitly indicates the mapping being performed (taking $x$ to $f(y)$).  The subsequent proof steps that are required are simply proofs by assumption.  Specifically, the original `Forall` expression must be true, the "instance" expression must be in the domain $S$, and the condition $Q(x)$ must apply when $x$ is replaced with the "instance" expression.  If any of these are not known to be true under the provided assumptions, this step will fail.

In this first case, it is unable to prove the original `Forall` expression even after attempting to perform automation.

In [10]:
from proveit import ProofFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[1:])
except ProofFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Unable to prove forall_{x in S | Q(x),R(x)} P(x) assuming {f(y) in S, Q(f(y)), R(f(y))}: Unable to conclude automatically; the domain has no 'foldAsForall' method.


The automation checks to see of the $S$ **Expression** has a `foldAsForall` method that would automate a proof for universal quantification over $S$.  Since $S$ is a simple **Variable** object, no such automation exists.  Next we'll see what happens when there the "instance" expression is not in the $S$ domain.

In [11]:
from proveit import SpecializationFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:1]+assumptions[2:])
except SpecializationFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x),R(x)} P(x), Q(f(y)), R(f(y))}: Unmet specialization condition: f(y) in S


Finally, we demonstrate the case when one of the conditions is not met.

In [12]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:2]+assumptions[3:])
except SpecializationFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x),R(x)} P(x), f(y) in S, R(f(y))}: Unmet specialization condition: Q(f(y))


In [13]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:3])
except SpecializationFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x),R(x)} P(x), f(y) in S, Q(f(y))}: Unmet specialization condition: R(f(y))


Also note that you cannot *specialize* a variable that is not one of the `Forall` instance variables.

In [14]:
basicForallExpr

forall_{x in S | Q(x),R(x)} P(x)

In [15]:
try:
    basicForallExpr.specialize({x:fy, Q:R}, assumptions=assumptions)
except SpecializationFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Proof step failed assuming {forall_{x in S | Q(x),R(x)} P(x), f(y) in S, Q(f(y)), R(f(y)), forall_{x in S | Q(x),R(x)} P(x)}: May only specialize instance variables of directly nested Forall operations


You can *relabel* **Variable**s that are not `Forall` instance variables.  Later we will show that you can *relabel* and *specialize* simultaneously.  You can also *specialize* multiple levels of `Forall` operations simultaneously which is why the previous error message mentions "nested Forall operations". 

### Universal quantification without a domain

It is not necessary to specify a domain in a `Forall` **Expression**.  For example, the condition(s) may provide sufficient restrictions for the universal quantification.  Also, any number of conditions may be specified (including no conditions).

In [16]:
noDomainForallExpr = Forall(x, Px, conditions=[Qx])
noDomainForallExpr

forall_{x | Q(x)} P(x)

In [17]:
print noDomainForallExpr.domain

None


In [18]:
noDomainForallExpr.exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x0000000004558DC8>

In [19]:
noDomainForallExpr.specialize({x:fy}, assumptions=[noDomainForallExpr, Operation(Q, fy)])

{[forall_{x | Q(x)} P(x)],Q(f(y))} |= P(f(y))

### Lambda scope restrictions

In `tutorial01_core_expr`, we noted scoping restrictions that apply to **Lambda** expressions in the context of *expression substitution*.  That restriction carries over to *specialization* and is very important.  Consider the following example.

In [20]:
from proveit.common import Pxy, y, fy
from proveit.logic import Exists, NotEquals
forallExistsExpr = Forall(x, Exists(y, NotEquals(x, y)))
forallExistsExpr

forall_{x} [exists_{y} (x != y)]

We are using `Exists` ($\exists$) and `Equals` ($\neq$) which do not have the special core status that `Forall` has and are discussed in `logic` package tutorials.  We are just using them to make our point below more clear.  Just note that `Exists` has the same kind of structure as $\forall$ and specifically acts as an **Operation** on an **Expression** with a **Lambda** function.

In [21]:
Exists(y, Pxy).exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x000000000459B088>

If we try to specialize $x$ as $y$ in `nestedForall`, this will fail:

In [22]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:y}, assumptions={forallExistsExpr})
except ScopingViolation as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Must not make substitution with reserved variables  (i.e., parameters of a Lambda function)


This should fail.  We cannot derive $\exists_y y \neq y$ by assuming $\forall_{ x } \left[\exists_y x \neq y \right]$.  The former is a stronger statement.  We chose this example, in fact, because the latter can be argued as typically true but the former is never true using reasonable definitions.  Where this goes wrong is in violating the scope of $\exists_y$.  It is introducing $y$ as a new **Variable** within the sub-expression $\exists_y x \neq y$.  This label is off limits to $x$ which is quantified outside of this sub-expression.  We can specialize $x$ to whatever we want as long as we respect these scoping restrictions.  It is not simply $y$ that is off limits; all expressions involving $y$ are off limits:

In [23]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:fy}, assumptions={forallExistsExpr})
except ScopingViolation as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Must not make substitution with reserved variables  (i.e., parameters of a Lambda function)


It should also be noted that within a scope, a **Variable** may be reused with a different meaning.  This should generally be avoided as it makes **Expression**s unclear, but the functionality should be well-defined in case it ever happens.  If this happens, we treat it as a distinct **Variable** from anything outside of the scope (that just happens to have the same name).  It can be confusing and should be avoided, but it is well-defined.  For example,

In [24]:
from proveit.logic import And
redundantInstanceVarExpr = Forall(x, And(Px, Forall(x, Qx)))
redundantInstanceVarExpr

forall_{x} (P(x) and [forall_{x} Q(x)])

In [25]:
# specializing the outer x does not and should not change the inner x which is treated as a distinct Variable
redundantInstanceVarExpr.specialize({x:fy}, assumptions={redundantInstanceVarExpr})

{[forall_{x} (P(x) and [forall_{x} Q(x)])]} |= P(f(y)) and [forall_{x} Q(x)]

### Operand, operator, or operation specialization

*Specializing* different parts of an **Operation** works essentially the same way as it does with *expression substition*.

We will use the `substitution` axiom of `proveit.logic.equality` for demonstrations in this section out of convenience.  Axioms and the `proveit.logic` package will be discussed in more detail later.  For now, we note that axioms (and theorems) are taken to be true statements without proof as you can see below.

In [26]:
from proveit.logic.equality._axioms_ import substitution
substitution

|= forall_{f, x, y | (x = y)} (f(x) = f(y))

In [27]:
substitution.proof()

<proveit._core_.proof.Axiom instance at 0x0000000004837688>

Let's grab the $x=y$ condition for use below.

In [28]:
x_eq_y = substitution.conditions[0]
x_eq_y

x = y

Operator and operand *specialization* is straightforward:

In [29]:
from proveit.common import f, g
operatorSubstitution = substitution.specialize({f:g}, assumptions=[x_eq_y])
operatorSubstitution

{(x = y)} |= g(x) = g(y)

In [30]:
operatorSubstitution.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004975B48>

Note that $x$ and $y$ mapped to themselves by default.  When a mapping is not specified, the default is to map the **Variable** to itself.

In [31]:
from proveit.logic import Equals
from proveit.common import a, b
a_eq_b = Equals(a, b)
operandSubstitution = substitution.specialize({x:a, y:b}, assumptions=[a_eq_b])
operandSubstitution

{(a = b)} |= f(a) = f(b)

In [32]:
operandSubstitution.proof()

<proveit._core_.proof.Specialization instance at 0x000000000495BEC8>

Operation substitution can be done explicitly via a **Lambda** expression just as we saw with *expression substitution*.  The **Lambda** expression is not literally substituted in; rather, the function that it represents is applied as the operation.

In [33]:
from proveit import Lambda
from proveit.number import Add
operationSubstitution = substitution.specialize({f:Lambda(x, Add(x, a))}, assumptions=[x_eq_y])
operationSubstitution

{(x = y)} |= (x + a) = (y + a)

In [34]:
operationSubstitution.proof()

<proveit._core_.proof.Specialization instance at 0x00000000049F1748>

An alternative way to specify an operation substitution is to map the operation applied to a **Variable** onto an expression that uses this same **Variable**.  In this example, mapping $f(x)$ to $x + a$.  This will be internally translated to the same **Lambda** expression as before: $x \mapsto x + a$.

In [35]:
from proveit.common import fx
operationSubstitution2 = substitution.specialize({fx:Add(x, a)}, assumptions=[x_eq_y])
operationSubstitution2

{(x = y)} |= (x + a) = (y + a)

The proof is exactly the same as before.

In [36]:
operationSubstitution2.proof()

<proveit._core_.proof.Specialization instance at 0x00000000049FCF88>

### Specializing multiple levels simultaneously

When `Forall` operations are nested, the universal quantifications may be specialized separately.  For example:

In [37]:
from proveit.number import Less
from proveit.common import z, Pxyz
nestedForall = Forall(x, Forall(y, Forall(z, Pxyz, conditions=[Less(z, Add(x, y))])))
nestedForall

forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]

In [38]:
nestedForallSpec1 = nestedForall.specialize(assumptions=[nestedForall])
nestedForallSpec1

{[forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]]} |= forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]

In [39]:
nestedForallSpec2 = nestedForallSpec1.specialize()
nestedForallSpec2

{[forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]]} |= forall_{z | (z < (x + y))} P(x,y,z)

In [40]:
nestedForallSpec3 = nestedForallSpec2.specialize(assumptions=[nestedForallSpec2.conditions[0]])
nestedForallSpec3

{(z < (x + y)),[forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]]} |= P(x,y,z)

In [41]:
nestedForallSpec3.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004A8F388>

But for the sake of convenience and efficiency, **Prove-It** also allows nested `Forall` operations to be specialized simultaneously:

In [42]:
assumptions = ExpressionList(nestedForall, nestedForallSpec2.conditions[0])
nestedForallSimultaneousSpec = nestedForall.specialize({z:z}, assumptions=assumptions)
nestedForallSimultaneousSpec

{[forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]],(z < (x + y))} |= P(x,y,z)

We just need to include an explicit mapping for an inner quantified **Variable**.  So we indicated above that we want to map $z$ to $z$ even though this is typically the default in order to force it to specialize all three `Forall` operations simultaneously.  The proof is shorter, doing a single all-in-one *specialization*:

In [43]:
nestedForallSimultaneousSpec.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004A8F5C8>

In the mapping that is indicated below step 0, enumerated set notation (with curly braces) is used to separate the mappings at different levels.  In this way, there can be no ambiguity.  Simultaneous *specialization* can be done for any number of nested levels.

### Specializing and relabeling simultaneously

It is also possible to *relabeling* and *specialization* (over any number of nested levels) in one step.  For example: 

In [44]:
nestedForallSpecAndRelab = nestedForall.specialize(specializeMap={y:y}, relabelMap={z:a}, assumptions=[nestedForall])
nestedForallSpecAndRelab

{[forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]]]} |= forall_{a | (a < (x + y))} P(x,y,a)

The `relabelMap` is specified separately from the `specializeMap` to be unambiguous.

In [45]:
nestedForallSpecAndRelab.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004A7F808>

The *relabeling* map is always shown after the last semicolon ';' in the *specialization* map.  When any *specialization* occurs, the "step type" is labeled "specialization".  If there is only *relabeling*, the "step type" will indicate "relabeling". 

You are not allowed to specify that the same **Variable** is to be *specialized* and *relabeled*.

In [46]:
from proveit import RelabelingFailure
try:
    nestedForall.specialize({y:y}, {y:a}, assumptions=assumptions)
except RelabelingFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Proof step failed assuming {forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]], z < (x + y)}: Cannot relabel using assumptions that involve any of the relabeling variables


As noted in the previous tutorial chapter, relabeling has another important limitation.  You cannot relabel something using assumptions that involve any of the relabeling variables.  For example, we cannot relabel $P$ to $R$ in `nestedForall` while assuming `nestedForall`.  

In [47]:
from proveit import RelabelingFailure
try:
    nestedForall.specialize({y:y}, {P:R}, assumptions=assumptions)
except RelabelingFailure as e:
    print "EXPECTED ERROR:", e

EXPECTED ERROR: Proof step failed assuming {forall_{x} [forall_{y} [forall_{z | (z < (x + y))} P(x,y,z)]], z < (x + y)}: Cannot relabel using assumptions that involve any of the relabeling variables


### Universal quantification over multiple variables

Rather than nesting `Forall` operations, you can quantify over multiple instance variables for a more succinct expression.

In [48]:
multiVarForall = Forall((x, y), Pxy, domain=S)
multiVarForall

forall_{x, y in S} P(x,y)

In [49]:
assumptions = [multiVarForall, InSet(x, S), InSet(y, S)]
multiVarForallSpec = multiVarForall.specialize(assumptions=assumptions)
multiVarForallSpec

{[forall_{x, y in S} P(x,y)],(x in S),(y in S)} |= P(x,y)

In [50]:
multiVarForallSpec.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004AA4288>

If you attempt to use the same **Variable** multiple times in the list of instance variables, you will get an error.

In [51]:
try:
    Forall((x, x), Px)
except ValueError as e:
    print 'EXPECTED ERROR:', e

EXPECTED ERROR: Lambda parameters Variables must be unique with respect to each other.


### Universal quantification over an unspecified number of variables

Using **MultiVariable**s, we can express universal quantification over an unspecified number of variables.  Depending upon how the **MultiVariable** is used in the instance expression, we may *specialize* it to an **ExpressionList** (if it is used within **Etcetera** sub-expressions), or an **ExpressionTensor** (if it is used within **Block** sub-expressions).

The following example uses a theorem from `proveit.logic.boolean.disjunction`.  It states, in English, that the `Or` (disjunction) operation acting an any number of operands is false if all of the operands are false.

In [52]:
from proveit.logic.boolean.disjunction._theorems_ import notOrIfNotAny
notOrIfNotAny

|= forall_{A_{*} in BOOLEANS |  ..,not(A_{*}),.. } not( .. or A_{*} or .. )

In [53]:
notOrIfNotAny.exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x0000000004BE3788>

Note that the instance variable (the parameter of the `imap`) is a **MultiVariable**.  The occurrences of this **MultiVariable** in the `iexpr` and `conds` are within **Etcetera** sub-expressions.  Below, we *specialize* this to an instance with three operands.

In [54]:
from proveit.common import Amulti, c
from proveit.logic import Not
notOrIfNotAnySpec = notOrIfNotAny.specialize({Amulti:[a, b, c]}, assumptions=[Not(a), Not(b), Not(c)])
notOrIfNotAnySpec

{not(a),not(b),not(c)} |= not(a or b or c)

In [55]:
notOrIfNotAnySpec.proof()

<proveit._core_.proof.Specialization instance at 0x0000000004BEE848>

Let us look at the expression information for this specialized instance to show that the **MultiVariable** has been substituted in a seemless manner.  Specifically, the `Or` operation has the $(a, b, c)$ **ExpressionList** operands.

In [56]:
notOrIfNotAnySpec.exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x000000000603E248>

It is possible to *specialize* the **MultiVariable** with zero expressions.

In [57]:
notOrIfNotAny.specialize({Amulti:[]})

|= not(\left[or
ight]())

We must format this using the $[\lor]$ operator.  This statement is valid and self-consistent because we define $[\lor]$ acting on zero operands to be `FALSE`:

In [58]:
from proveit.logic.boolean.disjunction._axioms_ import emptyDisjunction
emptyDisjunction

|= \left[or
ight]() = FALSE

It is also possible to include a **MultiVariable** in the *specialization*:

In [59]:
from proveit import Etcetera
from proveit.logic import inBool
from proveit.common import Bmulti
assumptions = [Not(a), Etcetera(Not(Bmulti)), inBool(a), Etcetera(inBool(Bmulti))]
notOrIfNotAnySpec2 = notOrIfNotAny.specialize({Amulti:(a, Bmulti)}, assumptions=assumptions)
notOrIfNotAnySpec2

{not(a), ..,not(B_{*}),...,(B_{*} in BOOLEANS),.. } |= not(a or  .. or B_{*} or .. )

Generalization
========

*Generalization* is the "inverse" of *specialization* just as *hypothetical reasoning* was the "inverse" of *modus ponens*.  We can write these derivation rules in a manner that makes this relationship clear (just as we did for *hypothetical reasoning* and *modus ponens*):

Specialization: 
$\begin{array}{c}
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x) \\
\hline
\left \{\clubsuit(x) \in S,~Q(\clubsuit(x)) \right \} \boldsymbol{\vdash} P(\clubsuit(x))
\end{array}$

Generalization: 
$\begin{array}{c}
\left \{ x \in S,~Q(x) \right \} \boldsymbol{\vdash} P(x) \\
\hline
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x)
\end{array}$


There is some asymmetry.  $\clubsuit(x)$ here is meant to represent any **Expression** involving $x$.  That is, $x$ may be *specialized* to any **Expression**, as long as the condition and domain requirements are satisfied for that expression.  However, *generalization* only applies to an unbound **Variable**.  In Prove-It, an unbound **Variable** is regarded as an "arbitrary" variable.  Essentially, it is implicitly universally quantified.  Recall that *modus ponens* converts an explicit antecedent to an implicit assumption and *hypothetical reasoning* does the opposite.  Similarly, *specialization* converts an explicit universal quantification to implicit arbitrary variables and *generalization* does the opposite.  The reason for having the explicit and implicit forms is much the same as it was for the antecedent versus assumption.  The explicit form allows nesting but the implicit form provides direct access to the instance expression.  Furthermore, explicit universal quantification offers the power of being able to *specialize* an instance variable to an arbitrary **Expression**.

The above derivation rules are expressed for a single **Variable**.  Such rules apply more generally to any number of **Variable** and/or **MultiVariable** instance variables.

Our following examples will start from one of the derived *specialization* instances above.  Specifically:

In [60]:
notOrIfNotAnySpec

{not(a),not(b),not(c)} |= not(a or b or c)

First, we try to *generalize* this **KnownTruth** for all instances of $a$, $b$, and $c$ without any conditions or domain restrictions:

In [61]:
from proveit import GeneralizationFailure
try:
    notOrIfNotAnySpec.generalize((a, b, c))
except GeneralizationFailure as e:
    print 'EXPECTED ERROR:', e

EXPECTED ERROR: Unable to prove forall_{a, b, c} not(a or b or c) assuming {not(c), not(b), not(a)}: Cannot generalize using assumptions that involve any of the new forall variables (except as assumptions are eliminated via conditions or domains)


This fails because the assumptions of the original **KnownTruth** involve the same **Variable**s that we are trying to *generalize* over.  That is not allowed because universal quantification introduces a new scope for the instance variables and the assumptions can no longer apply to these variables in the new scope.  If, however, these assumptions are introduced as conditions of the new universal quantification, then we no longer need to retain them as assumptions.  Those assumptions will be absorbed into the universal quantification conditions.

In [62]:
notOrIfNotAnySpec.generalize((a, b, c), conditions=[Not(a), Not(b), Not(c)])

|= forall_{a, b, c | not(a),not(b),not(c)} not(a or b or c)

Adding additional restrictions, such as a domain and/or extra conditions, only makes the statement weaker and is therefore allowed:

In [63]:
notOrIfNotAnySpec.generalize((a, b, c), conditions=[Not(a), Not(b), Not(c)], domain=S)

|= forall_{a, b, c in S | not(a),not(b),not(c)} not(a or b or c)

In [64]:
notOrIfNotAnySpec.generalize((a, b, c), conditions=[Not(a), Not(b), Not(c), Qx], domain=S)

|= forall_{a, b, c in S | not(a),not(b),not(c),Q(x)} not(a or b or c)

It is also possible to create multiple levels of nested `Forall` operations in one step with possibly different domains.  Simply provide a list of lists of **Variable**s as the first argument to `generalize` and a corresponding list of `domains`.  The conditions are applied at the outermost level possible (as soon as all of the relevant variables have been introduced) while retaining the order that the `conditions` were supplied.

In [65]:
notOrIfNotAnyNested = notOrIfNotAnySpec.generalize([[a], [b], [c]], domains=(P, R, S), conditions=[Not(a), Not(b), Not(c)])
notOrIfNotAnyNested

|= forall_{a in P | not(a)} [forall_{b in R | not(b)} [forall_{c in S | not(c)} not(a or b or c)]]

The generalization with multiple leves of nested `Forall` operations takes one step in the proof:

In [66]:
notOrIfNotAnyNested.proof()

<proveit._core_.proof.Generalization instance at 0x000000000604BAC8>

This next example changes the order of the conditions which affects when they are introduced.

In [67]:
notOrIfNotAnySpec.generalize([[a], [b], [c]], domains=(P, R, S), conditions=[Not(a), Not(c), Not(b)])

|= forall_{a in P | not(a)} [forall_{b in R} [forall_{c in S | not(c),not(b)} not(a or b or c)]]

The number of provided `domains` must match the number of nested levels of `Forall` to be introduced.

In [68]:
try:
    notOrIfNotAnySpec.generalize([[a], [b], [c]], domains=(P, R), conditions=[Not(a), Not(b), Not(c)])
except ValueError as e:
    print 'EXPECTED ERROR:', e

EXPECTED ERROR: The number of forall variable lists and new domains does not match: 3 vs 2


When *generalizing* over a single **Variable**, the first argument may be just that **Variable** rather than a list or tuple:

In [69]:
notOrIfNotAnySpec.generalize(a, conditions=[Not(a)])

{not(c),not(b)} |= forall_{a | not(a)} not(a or b or c)

We may only *generalize* over **Variable**s and/or **MultiVariable**s, however:

In [70]:
try:
    notOrIfNotAnySpec.generalize(Qx, conditions=[Not(a)])
except ValueError as e:
    print 'EXPECTED ERROR:', e

EXPECTED ERROR: Must supply generalize with a Variable or list of Variables


In [71]:
try:
    notOrIfNotAnySpec.generalize([a, Qx], conditions=[Not(a)])
except ValueError as e:
    print 'EXPECTED ERROR:', e

EXPECTED ERROR: Forall variables of a generalization must be Variable objects


Let us also demonstrate *generalization* over a **MultiVariable**.  For this example, we will use the previous *specialization* instance that generated a **MultiVariable** as an implicit arbitrary multi-variable that we well next convert back to an explicit universal quantification.

In [74]:
notOrIfNotAnySpec2

{not(a), ..,not(B_{*}),...,(B_{*} in BOOLEANS),.. } |= not(a or  .. or B_{*} or .. )

In [72]:
notOrIfNotAnySpec2 # HOW DID THE ETCETERA IN LIST FORMATTING GET BROKEN?

In [73]:
from proveit.logic import Booleans
notOrIfNotAnySpec2.generalize((a, Bmulti), conditions=[Not(a), Etcetera(Not(Bmulti))], domain=Booleans)

This could have been proven in one step from `notOrIfNotAny` using *relabeling*, but this demonstrates going back and forth from an explicit universal quantification to an implicit arbitrary multi-variable and back in a self-consistent manner. 

# Next chapter: <a href="tutorial06_theorem_proving.ipynb">Theorem Proving</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>

In [1]:
from proveit.number import Fraction
from proveit.common import x, y, A
from proveit.logic import Implies

In [2]:
Fraction(x, y).prove([A, Implies(A, Fraction(x, y))])

{(A => (x // y)),A} |= x // y

In [3]:
Fraction(x, y).exprInfo()

<proveit._core_.expression.expr_info.ExpressionInfo instance at 0x00000000044BD488>